In [6]:
import cvxpy
import numpy as np

In [11]:
def power_utility(x: float, gamma: float) -> float:
    """Constant RRA utility function. Optimal for investments (source Jorgen).

    Args:
        x (float): variable value
        gamma (float): risk aversion parameter

    Raises:
        ValueError: if gamma is greater than 1.

    Returns:
        float: utility of x.
    """

    if gamma > 1:
        raise ValueError("Value of gamma is larger than 1.")
    if gamma != 0:
        return x ** gamma / gamma
    else:
        return np.log(x)